In [3]:
import pybel
from pybel.io.jupyter import to_jupyter
from utils import *
import seaborn as sns
import pandas as pd

In [8]:
chembl = pd.read_excel("data/normalized_data/Marburg_virus_KG/Marburg_smiles.xlsx")

In [9]:
chembl=chembl['Molecule ChEMBL ID']

In [10]:
chembl2mech = RetMech(chembl)

Retrieving mechanisms from ChEMBL: 100%|██████████| 57/57 [00:21<00:00,  2.60it/s]


In [12]:
chembl2dis = RetDrugInd(chembl)

Retrieving diseases from ChEMBL: 100%|██████████| 57/57 [00:23<00:00,  2.44it/s]


In [14]:
chembl2act = RetAct(chembl) 

Retrieving bioassays from ChEMBL: 100%|██████████| 57/57 [00:36<00:00,  1.57it/s]


In [15]:
chembl2act

{'CHEMBL226327': [{'assay_chembl_id': 'CHEMBL904964',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '7.96',
   'target_chembl_id': 'CHEMBL206',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '10.9'},
  {'assay_chembl_id': 'CHEMBL904965',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '7.79',
   'target_chembl_id': 'CHEMBL242',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '16.3'}],
 'CHEMBL28211': [{'assay_chembl_id': 'CHEMBL839610',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '7.52',
   'target_chembl_id': 'CHEMBL287',
   'target_organism': 'Homo sapiens',
   'type': 'Ki',
   'value': '30.0'}],
 'CHEMBL226329': [{'assay_chembl_id': 'CHEMBL904964',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '7.71',
   'target_chembl_id': 'CHEMBL206',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
  

In [16]:
prtn_as_chembl = Ret_chembl_protein(chembl2act) + Ret_chembl_protein(chembl2mech)
prtn_as_chembl = set(prtn_as_chembl)
prtn_as_chembl = list(prtn_as_chembl)
len(prtn_as_chembl)

16

In [17]:
chembl2uprot = chembl2uniprot(prtn_as_chembl)

Populating ChEMBL data for human proteins: 100%|██████████| 16/16 [00:00<00:00, 285.48it/s]


In [19]:
chembl2uprot

{'CHEMBL228': [{'xref_id': 'R-HSA-380615',
   'xref_name': 'Serotonin clearance from the synaptic cleft.',
   'xref_src_db': 'Reactome'},
  {'accession': 'P31645'},
  {'component_synonym': 'SLC6A4', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL2311238': [{'accession': 'Q9BY08'},
  {'component_synonym': 'EBPL', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL2364163': [{'xref_id': 'R-HSA-166665',
   'xref_name': 'Terminal pathway of complement.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-174577',
   'xref_name': 'Activation of C3 and C5.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-375276',
   'xref_name': 'Peptide ligand-binding receptors.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-418594',
   'xref_name': 'G alpha (i) signalling events.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-977606',
   'xref_name': 'Regulation of Complement cascade.',
   'xref_src_db': 'Reactome'},
  {'accession': 'P01031'},
  {'component_synonym': 'C5', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL

In [20]:
chembl2act = chembl2gene2path(chembl2uprot, chembl2act)
chembl2mech = chembl2gene2path(chembl2uprot, chembl2mech)

In [21]:
mburg_graph = pybel.BELGraph(name='Marburg Graph', version="0.0.1")

In [22]:
mburg_graph = chem2moa_rel(chembl2mech, 'HGNC', mburg_graph)
mburg_graph = chem2dis_rel(chembl2dis, mburg_graph)
mburg_graph = chem2act_rel(chembl2act, 'HGNC', mburg_graph)
mburg_graph = gene2path_rel(chembl2uprot, 'HGNC', mburg_graph)

Adding bioassay edges to BEL: 100%|██████████| 8/8 [00:00<00:00, 228.55it/s]


In [23]:
pybel.dump(mburg_graph, 'data/normalized_data/Marburg_virus_KG//Marburg.bel.pickle')

In [24]:
to_jupyter(mburg_graph)

<IPython.core.display.Javascript object>

In [25]:
#Fetch all proteins from the graph
chemblProt = Ret_uprotid(chembl2uprot)
chemblProt 

['P31645',
 'Q9BY08',
 'P01031',
 'P62508',
 'P03372',
 'Q92731',
 'P08684',
 'Q99720',
 'Q92731',
 'P20813',
 'Q01959',
 'P11712',
 'P23975',
 'P33261',
 'Q06278',
 'P08183']

In [26]:
uprots_ext = ExtractFromUniProt(chemblProt)

In [27]:
uprots_ext

{'P31645': {'Disease': {},
  'Reactome': {'Serotonin clearance from the synaptic clef': 'R-HSA-380615'},
  'Function': {'actin filament binding': 'GO:0051015',
   'cocaine binding': 'GO:0019811',
   'identical protein binding': 'GO:0042802',
   'metal ion binding': 'GO:0046872',
   'monoamine transmembrane transporter activity': 'GO:0008504'},
  'BioProcess': {'brain morphogenesis': 'GO:0048854',
   'cellular response to cGMP': 'GO:0071321',
   'cellular response to retinoic acid': 'GO:0071300',
   'circadian rhythm': 'GO:0007623',
   'memory': 'GO:0007613'},
  'Gene': 'SLC6A4'},
 'Q9BY08': {'Disease': {},
  'Reactome': {},
  'Function': {'cholestenol delta-isomerase activity': 'GO:0047750'},
  'BioProcess': {'sterol metabolic process': 'GO:0016125'},
  'Gene': 'EBPL'},
 'P01031': {'Disease': {'Complement component 5 deficiency (C5D)': '609536'},
  'Reactome': {'Terminal pathway of complemen': 'R-HSA-166665',
   'Activation of C3 and C': 'R-HSA-174577',
   'Peptide ligand-binding recep

In [28]:
mburg_graph = uniprot_rel(uprots_ext, 'HGNC', mburg_graph)

In [29]:
to_jupyter(mburg_graph)

<IPython.core.display.Javascript object>

In [40]:
mburg_prot = pd.read_excel('data/normalized_data/Marburg_virus_KG/uniprot-download_true_fields_accession_2Cid_2Cgene_names_2Cgene_prim-2022.08.12-08.21.53.55.xlsx')

C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [41]:
mburg_prot.head(5)

,From,Entry,Entry Name,Gene Names,Gene Names (primary)
0,PSMB5,P28074,PSB5_HUMAN,PSMB5 LMPX MB1 X,PSMB5
1,PSMB1,P20618,PSB1_HUMAN,PSMB1 PSC5,PSMB1
2,MAPT,P10636,TAU_HUMAN,MAPT MAPTL MTBT1 TAU,MAPT
3,CYP3A4,P08684,CP3A4_HUMAN,CYP3A4 CYP3A3,CYP3A4
4,CYP2B6,P20813,CP2B6_HUMAN,CYP2B6,CYP2B6


In [34]:
mburg_prot=mburg_prot['Entry']

In [35]:
mburg_prot_ext = ExtractFromUniProt(mburg_prot)

In [37]:
mburg_graph = uniprot_rel(mburg_prot_ext, 'HGNC', mburg_graph)

In [38]:
to_jupyter(mburg_graph)

<IPython.core.display.Javascript object>

In [39]:
pybel.dump(mburg_graph, 'data/normalized_data/Marburg_virus_KG//Marburg_V1.0.bel.pickle')

In [42]:
infile = open('data/normalized_data/Marburg_virus_KG//Marburg_V1.0.bel.pickle','rb')
mburg_graph = pickle.load(infile)
infile.close()

In [44]:
mburg_graph.summarize

Name,Marburg Graph
Version,0.0.1
Nodes,605
Namespaces,8
Edges,1776
Annotations,0
Citations,2
Authors,0
Components,3
Warnings,0
Network Density,0.00486


In [45]:
## to Neo4j
# Exporting graph to Neo4J
import py2neo
node_map = {}

neo_connection = py2neo.Graph('bolt://localhost:7687', auth=('neo4j', 'itmp'),name='marburg')
tx = neo_connection.begin()

nodes = list(mburg_graph)

for node in tqdm(nodes, desc="nodes"):
    attrs = {"namespace": node.namespace}

    if node.name and node.identifier:
        attrs["name"] = node.name
        attrs["identifier"] = node.identifier
    elif node.identifier and not node.name:
        attrs["name"] = node.identifier
    elif node.name and not node.identifier:
        attrs["name"] = node.name

    node_map[node] = py2neo.Node(node.function, **attrs)

    tx.create(node_map[node])

edges = mburg_graph.edges(keys=True, data=True)

for u, v, key, node in tqdm(edges, desc="edges"):
    rel_type = node['relation']

    d = node.copy()
    del d['relation']

    attrs = {}

    annotations = d.pop('annotations', None)
    if annotations:
        for annotation, values in annotations.items():
            attrs[annotation] = list(values)

    citation = d.pop('citation', None)
    if citation:
        attrs['citation'] = citation.curie

    if 'evidence' in d:
        attrs['evidence'] = d['evidence']

    rel = py2neo.Relationship(node_map[u], rel_type, node_map[v], key=key, **attrs)
    tx.create(rel)
    
tx.commit()

edges: 100%|██████████| 1776/1776 [00:09<00:00, 189.95it/s]
C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_1992\2110126565.py:51: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()
